# Test pandas_ta indicators

Import libraries

In [ ]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from scipy.stats import linregress

Get USDJPY hourly data, starting 2021-01-01 to 2023-01-01

In [ ]:
data = pd.read_csv("USDJPY_data.csv")
data.head()

Create and drop a dataframe that contains all the days with 0 volume, those are non trading days

In [ ]:
data_with_zeros = data[data.Volume == 0].index
data.drop(data_with_zeros, inplace=True)
data.loc[data.Volume == 0]

Setup some indicators

In [ ]:
data['ATR']     = data.ta.atr(length=20)
data['RSI']     = data.ta.rsi()
data['Average'] = data.ta.midprice(length=1) #midprice??
data['MA40']    = data.ta.sma(length=40)
data['MA80']    = data.ta.sma(length=80)
data['MA160']   = data.ta.sma(length=160)

def get_slope(array):
    y = np.array(array)
    x = np.arange(len(y))
    
    slope, intercept, r_value, p_value, std_err = linregress(x,y)
    
    return slope

back_rolling_amount = 6

data['MA40_slope']    = data['MA40'].rolling(window=back_rolling_amount).apply(get_slope, raw=True)
data['MA80_slope']    = data['MA80'].rolling(window=back_rolling_amount).apply(get_slope, raw=True)
data['MA160_slope']   = data['MA160'].rolling(window=back_rolling_amount).apply(get_slope, raw=True)
data['Average_slope'] = data['Average'].rolling(window=back_rolling_amount).apply(get_slope, raw=True)
data['RSI_slope']     = data['RSI'].rolling(window=back_rolling_amount).apply(get_slope, raw=True)

data.tail()

Set take profit and stop loss

In [5]:
pipdiff = 450*1e-5
SLTPRatio = 2

In [ ]:
def mytarget(barsupfront, data):
    length = len(data)
    high = list(data['High'])
    low = list(data['Low'])
    close = list(data['Close'])
    open = list(data['Open'])
    trendcat = [None] * length
    
    for line in range (0, length-barsupfront-2):
        value_open_low = 0
        value_open_high = 0
        